#### Шаги реализации:
Определение индивида для мета-алгоритма:
Индивид будет представлять гиперпараметры первичного алгоритма. Например, индивид может быть списком вида [cxpb, mutpb, ngen, population_size].

#### Функция оценки:
Функция оценки будет запускать первичный генетический алгоритм с заданными гиперпараметрами и возвращать какую-то метрику производительности, например, наилучшую приспособленность, достигнутую первичным алгоритмом.

#### Операторы скрещивания, мутации и отбора:
Для мета-алгоритма необходимо определить, как будут происходить скрещивание, мутация и отбор индивидов (наборов гиперпараметров).

#### Запуск мета-алгоритма:
Запустите мета-алгоритм и позвольте ему оптимизировать гиперпараметры первичного алгоритма.

#### Пример кода:
Этот код демонстрирует концепцию и требует дальнейшей адаптации под вашу конкретную задачу и первичный алгоритм.

В этом примере необходимо определить функцию **run_primary_ga** , которая будет запускать ваш первичный генетический алгоритм с заданными гиперпараметрами и возвращать его производительность.

In [13]:
# ЗАГРУЗКА БИБЛИОТЕК
import xlrd, xlwt       # модули для работы с файлами xls

import numpy as np # Библиотека работы с массивами
import matplotlib.pyplot as plt # Отрисовка изображений
import pandas as pd # Библиотека pandas
import os # Загружаем модуль os для поиска файлов в папках

from datetime import datetime # модуль для расчета разницы в датах
from datetime import timedelta

import random
from deap import base, creator, tools, algorithms
from functools import partial

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pyxlsb
import os
import keyboard

# Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline

In [27]:
# ИНИЦИАЦИЯ ПЕРЕМЕННЫХ
#путь к файлу графика ГРП

# Спрашиваем с каким графиком работать
# Или с нуля строим график из файла График_ГРП.xlsx
# Или оптимизируем файл График_ГРП_(оптимизированный).xlsx

FName ="График_ГРП_GA.xlsx"

Doobuchenie=False

basePath ='./'


skiprs=17                   # количество строк пропускаем при считывании листа ГРП
# Открываем рабочую книгу
wb = openpyxl.load_workbook(basePath+FName)
ws = wb['График ГРП']       # Выбираем рабочий лист (например, первый)

FirstDate = ws['A13'].value  # Считываем первый день месяца для расчета накопленного дебита 
LastDate = ws['A14'].value  # Считываем последний день месяца для расчета накопленного дебита
FlotsNumber=ws['A15'].value # количество флотов

# MinMaxcell=ws['A16'].value  # тип оптимизатора Min или Max
# MaxGen=ws['A17'].value                   # количество генераций поколений

# !!!!!!!!!!! данные для обучения !!!!!!!!!!!!!!!!
MinMaxcell='Max'  # тип оптимизатора Min или Max
MaxGen=10                   # количество генераций поколений

BestCount=30

wb.close()                  # Закрываем рабочую книгу

# считываем график из excel
gGRP=pd.read_excel(basePath + FName, sheet_name='График ГРП', usecols="B:AM", index_col=None,  skiprows=skiprs)

gGRP = gGRP.dropna(subset=['SKV'])   #удялем строки с пустыми значениями №СКВ, №КУСТ 
gGRP = gGRP.dropna(subset=['KUST'])

gGRP_Back=gGRP.copy() # копия на случай тренировок

# считываем таблицу со временем переезда между ЦДНГ
PereezdTimeBase=pd.read_excel(basePath + FName, sheet_name='переезды', usecols="A:K", index_col="ЦДНГ",  skiprows=0)

# PodvGRPgrapf=pd.read_excel(basePath + FName, sheet_name='подвески', usecols="A:S", index_col="PodvGRP",  skiprows=0)

# Переводим даты из формата excel в формат pandas !!!
gGRP.DateGotGRP = pd.to_datetime(gGRP.DateGotGRP)
gGRP.DateNachGRP = pd.to_datetime(gGRP.DateNachGRP)
gGRP.DateKonGRP = pd.to_datetime(gGRP.DateKonGRP)

# переводим нужные данные в число
gGRP.Qn_ozid = gGRP.Qn_ozid.apply(int)
gGRP.Nstadiy = gGRP.Nstadiy.apply(int)
gGRP.Tn_prop_stad = gGRP.Tn_prop_stad.apply(int)
gGRP.Time_stad = gGRP.Time_stad.apply(int)
gGRP.TimeGRP = gGRP.TimeGRP.apply(int)

# переводим некоторые значения в текст
# gGRP.FlotNumber = gGRP.FlotNumber.apply(str)

# Добавляем столбец с расчетом максимально возможного накопленного дебита 
gGRP["QnMAX"]=((LastDate-gGRP["DateKonGRP"]).dt.days-gGRP["TimeGRP"])*gGRP["Qn_ozid"]

print(gGRP.shape)

# модуль проверок пустых значений в ЦДНГ, ДатеГотовности к ГРП, Qн
if gGRP.CDNG.isnull().any():
    print ("Не во всех строка указан номер ЦДНГ")
    keyboard.read_key()
#     exit()
    
if gGRP.DateGotGRP.isnull().any():
    print ("Не во всех строках указана дата готовности к ГРП")
    keyboard.read_key()
#     exit()

if gGRP.Qn_ozid.isnull().any():
    print ("Не во всех строках указан ожидаемый дебит нефти")
    keyboard.read_key()
#     exit()

if gGRP.Kvaznost.isnull().any():
    print ("Не во всех строках указан коэффициент важности скважины. от 1 - самый низкий, 100 - самый высокий")
    keyboard.read_key()
#     exit()

# ищем и копируем в ActiveGRP строки из базы gGRP где сейчас стоят флота ГРП

# для корректной работы еще раз переовдим DateKonGRP в дату пандас
gGRP['DateKonGRP'] = pd.to_datetime(gGRP['DateKonGRP'])

# Находим минимальные значения для каждой группы в 'DateKonGRP' и уникальные в FlotNumber
grouped = gGRP.groupby('FlotNumber')['DateKonGRP'].idxmin()

# Получаем номера строк, удовлетворяющих условию
rows_to_keep = grouped.values

# Создаем ActiveGRP, содержащий строки, удовлетворяющие условию
ActiveGRP = gGRP.loc[rows_to_keep]

# удаляем из gGRP строки, удовлетворяющие условию
gGRP = gGRP.drop(rows_to_keep)

# очищаем даты начала окончания ГРП
gGRP.DateNachGRP=pd.NaT
gGRP.DateKonGRP=pd.NaT

print(f"gGRP.shape {gGRP.shape}")
print(f"ActiveGRP.shape {ActiveGRP.shape}")

ActiveGRP

# сортируем по дате готовности к ГРП

gGRP=gGRP.sort_values(by=['DateGotGRP','CDNG','Qn_ozid'], ascending=[True,True,False])


# преобразуем название действующих флотов в список
FlotsName=ActiveGRP["FlotNumber"].tolist()

# копируем в аналогичный список даты окончания ГРП
ActiveGRPend=ActiveGRP["DateKonGRP"].tolist()

# копируем в список номера ЦДНГ по текущим скважинам
ActiveGRPcdng=ActiveGRP["CDNG"].tolist()

tempGRPend=ActiveGRPend.copy()
tempGRPcdng=ActiveGRPcdng.copy()


(34, 39)
gGRP.shape (31, 39)
ActiveGRP.shape (3, 39)


In [28]:
# ФУНКЦИЯ РАСЧЕТА ОШИБКИ MAX - максимальный накопленный дебит
def evaluate (individual):           # функция  расчета максимального дебита от графика ГРП
    mm="Max"
    bot=individual
    QnTemp=0                             # начальный накопленный дебит нефти
    tempGRPend=ActiveGRPend.copy()       # заносим во времененную перменную даты окочания ГРП по рвущимся скважинам
    tempGRPcdng=ActiveGRPcdng.copy()     # заносим во времененную перменную номера ЦДНГ по рвущимся скважинам
    #tempGRPbegin=tempGRPend[0]
    
# цикл расчета накопленного дебита для данного графика флотов ГРП
    for i in range(0,len(bot)): 
            if bot[i]!=0:         # если номер флота не равен 0 то запускаем просчет ошибки 
            
                d=int(gGRP["TimeGRP"].iloc[i])                                        # время проведения ГРП на скважине

                CDNGactive=int(gGRP["CDNG"].iloc[i])                                  # номер ЦДНГ текущей скважины
                CDNGlast=tempGRPcdng[bot[i]-1]                                        # номер ЦДНГ прошлой скважины
                TimePereezd=PereezdTimeBase[CDNGlast][CDNGactive]                     # время переезда между ЦДНГ (сут)


        #       если дата готовности скважины к ГРП, позже даты освобождения флота
        #       начальной датой ГРП берем дату готовности СКВАЖИНЫ к ГРП
        #       в противном случай датой начала ГРП будет дата ОСВОБОЖДЕНИЯ ФЛОТА
                if gGRP["DateGotGRP"].iloc[i] < tempGRPend[bot[i]-1]:
                    tempGRPbegin=tempGRPend[bot[i]-1]
                    tempGRPend[bot[i]-1] = tempGRPbegin + timedelta(days=d + int(TimePereezd))       # обновляем дату готовности к ГРП флота bot[i]
                else:
                    tempGRPbegin=gGRP["DateGotGRP"].iloc[i]
                    tempGRPend[bot[i]-1] = tempGRPbegin + timedelta(days=d + int(TimePereezd)) # обновляем дату готовности к ГРП флота bot[i]

                if mm == "Max":
                    QnTemp+= (LastDate-tempGRPend[bot[i]-1]).days * gGRP["Qn_ozid"].iloc[i] * gGRP["Kvaznost"].iloc[i] # добавляем накопленную добычу с учетом коэффициента важности скважины
                elif mm == "Min":
                    timeD = tempGRPbegin - gGRP["DateGotGRP"].iloc[i]
                    QnTemp+= abs(timeD.days) + (-gGRP["Qn_ozid"].iloc[i] * gGRP["Kvaznost"].iloc[i]) #суммируем минимальное время ожидания флота коэффициента важности скважины
                            
                tempGRPcdng[bot[i]-1]=CDNGactive                               # обновляем номер ЦДНГ для текущего флота  
    return(QnTemp,)

In [50]:
def run_primary_ga (cx, mut, pop_size, ng=20):

    FlotsN=list(range(0, FlotsNumber+1))

    # функцию оптимизации берем из файла График ГРП
    # (Min - минимальное время ожидания флота ГРП, Max - максимальная накопленная добыча) 
    MinMax="Max"

    # Определяем минимизацию или максимизацию
    if MinMax == "Max":
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)
    elif MinMax == "Min":
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    SkvQuantity=gGRP.shape[0]  # количество скважин для перебора

    toolbox.register("attr_int", random.choice, FlotsN)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=SkvQuantity)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=0, up=FlotsNumber, indpb=0.1)  # Изменен диапазон мутации

    toolbox.register("select", tools.selTournament, tournsize=3)
    # toolbox.register("select", tools.selBest)

    toolbox.register("evaluate", evaluate)
#     toolbox.register("evaluate", partial(evaluate, mm=MinMax))  # Или MinMax="Min" в зависимости от того, что вам нужно

# ---------!!!!! запуск генеративной сети !!!!!!--------------
    pop = toolbox.population(n=int(pop_size))
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=cx, mutpb=mut, ngen=ng, stats=stats, halloffame=hof, verbose=False)
    
    Best=hof[0].fitness.values[0]
    

#     Best = RunGenereate(0.5, 0.4, 10, 500)   # cx, mut, ng, pop_size, MinMax
    return Best

In [18]:
run_primary_ga(0.5, 0.4, 500) # cx, mut, pop_size, ng=10

c:\users\bigman\appdata\local\programs\python\python38\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\users\bigman\appdata\local\programs\python\python38\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


108644.0

In [52]:
from deap import base, creator, tools, algorithms
import random

# Допустим, у нас уже есть функция для запуска первичного генетического алгоритма
# def run_primary_ga(cxpb, mutpb, ngen, population_size): ...

# Определение индивида для мета-алгоритма
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

#------здесь задаем данные гипер параметров для подбора-------------------------------
toolbox.register("attr_float", random.uniform, 0.1, 0.9)  # Для cxpb и mutpb
toolbox.register("attr_int", random.randint, 100, 800)  # Для ngen и population_size
#-------------------------------------------------------------------------------------

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_float, toolbox.attr_float, toolbox.attr_int), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate1(individual):
    print(f"cx, mut, pop_size =  {individual}")  # выводим гиперемараметры для просчета
    cxpb, mutpb, population_size = individual    # выводим гиперпараметры для обучения из созданного бота 
    
    # Запуск первичного GA с данными гиперпараметрами и возврат его лучшей приспособленности
    bestresult = run_primary_ga (cxpb, mutpb, population_size)
    print(bestresult)                            # выводим лучший результат по подобранным гиперпараметрам
    return bestresult,

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate1)

def main():
    pop = toolbox.population(n=50)    # !!!! указать кол-во популяций для просчета !!!!!!
    hof = tools.HallOfFame(3)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("max", max)


    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=50, 
                        stats=stats, halloffame=hof, verbose=True)
    
    return pop, hof

In [ ]:
# программа запуска алгоритма для подбора гиперпараметров сети 
pop, hof = main()
print("Лучшие гиперпараметры cxpb, mutpb, population_size:", hof[0])
print("Лучшее значение:", hof[0].fitness.values[0])

In [66]:
print("Лучшее значение:", hof[0].fitness.values[0])

print (run_primary_ga(1.1522749365764187, 0.13555863750682456, 562))
print (run_primary_ga(0.8039690222918632, 0.10096793123263802, 651))


Лучшее значение: 13178.0
13023.0
12875.0


In [ ]:
Лучшие гиперпараметры cxpb, mutpb, population_size: [0.8039690222918632, 0.10096793123263802, 651]
Лучшее значение: 12828.0

In [ ]:
Лучшие гиперпараметры cxpb, mutpb, population_size: [1.1522749365764187, 0.13555863750682456, 562]
Лучшее значение: 13178.0